In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



#postulantesEducacion = pd.read_csv('resources/tar/fiuba_1_postulantes_educacion.csv')
#postulantesGenero = pd.read_csv('resources/tar/fiuba_2_postulantes_genero_y_edad.csv')
vistas = pd.read_csv('resources/tar/fiuba_3_vistas.csv')
postulaciones = pd.read_csv('resources/tar/fiuba_4_postulaciones.csv')
#avisosOnline = pd.read_csv('resources/tar/fiuba_5_avisos_online.csv')
avisosDetalle = pd.read_csv('resources/tar/fiuba_6_avisos_detalle.csv')




In [264]:
#cargamos zips
avisosDetalleZIP = pd.read_csv('resources/zip/fiuba_6_avisos_detalle.csv')
avisosDetalleMissing = pd.read_csv('resources/zip/fiuba_6_avisos_missing.csv')

In [265]:
avisosDetallesCompletos = pd.concat([avisosDetalleZIP,avisosDetalleMissing])

In [259]:
testingSet = pd.read_csv('resources/test_final_100k.csv')

In [3]:
postulacionesMerged = pd.merge(postulaciones,avisosDetalle,on='idaviso',how='inner');

In [4]:
postulacionesMerged['datePostulacion'] = pd.DatetimeIndex(postulacionesMerged['fechapostulacion']).date
postulacionesMergedFiltered = postulacionesMerged.drop(['ciudad','mapacalle','idpais','fechapostulacion'],axis=1)

In [5]:
postulacionesMergedFiltered.columns

Index(['idaviso', 'idpostulante', 'titulo', 'descripcion', 'nombre_zona',
       'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',
       'denominacion_empresa', 'datePostulacion'],
      dtype='object')

In [5]:
postulacionesMerged['cantidadEmpresa'] = 1;
cantAvisosPorEmpresa = postulacionesMerged.groupby('denominacion_empresa').sum()
cantAvisosPorEmpresa = cantAvisosPorEmpresa.reset_index()
cantAvisosPorEmpresa = cantAvisosPorEmpresa[['denominacion_empresa','cantidadEmpresa']]

In [6]:
postulacionesConEmpresas = pd.merge(postulacionesMergedFiltered,cantAvisosPorEmpresa,how='inner',on='denominacion_empresa')

In [65]:
postulacionesConEmpresas.columns

Index(['idaviso', 'idpostulante', 'titulo', 'descripcion', 'nombre_zona',
       'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',
       'denominacion_empresa', 'datePostulacion', 'cantidadEmpresa'],
      dtype='object')

In [66]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

tfidfTransformer = TfidfTransformer()
countVec = CountVectorizer()

In [7]:
postulacionesFinal = postulacionesConEmpresas.drop('titulo',axis=1)
postulacionesFinal.drop(['descripcion','denominacion_empresa'],axis=1,inplace=True);
postulacionesFinal.drop('nombre_area',axis=1,inplace=True)

In [68]:
postulacionesFinal.head()

,idaviso,idpostulante,nombre_zona,tipo_de_trabajo,nivel_laboral,datePostulacion,cantidadEmpresa
0,1112248724,NjlD,Gran Buenos Aires,Full-time,Senior / Semi-Senior,2018-01-19,2318
1,1112248724,MV6eM8N,Gran Buenos Aires,Full-time,Senior / Semi-Senior,2018-01-30,2318
2,1112248724,5LzQkN,Gran Buenos Aires,Full-time,Senior / Semi-Senior,2018-01-15,2318
3,1112248724,8M2rOBz,Gran Buenos Aires,Full-time,Senior / Semi-Senior,2018-01-30,2318
4,1112248724,Dr6V3Da,Gran Buenos Aires,Full-time,Senior / Semi-Senior,2018-01-23,2318


In [8]:
postulacionesOHC = pd.concat([postulacionesFinal,pd.get_dummies(postulacionesFinal['nombre_zona'], prefix='zona')],axis=1)
postulacionesOHC = pd.concat([postulacionesOHC,pd.get_dummies(postulacionesFinal['tipo_de_trabajo'], prefix='tipo')],axis=1)
postulacionesOHC = pd.concat([postulacionesOHC,pd.get_dummies(postulacionesFinal['nivel_laboral'], prefix='nivel')],axis=1)

In [9]:
postulacionesOHC.drop(['nivel_laboral','tipo_de_trabajo','nombre_zona'],axis=1,inplace=True)

In [10]:
postulacionesOHC['sepostulo'] = 1;

In [55]:
postulacionesOHC.info

<bound method DataFrame.info of             idaviso idpostulante  cantidadEmpresa_x  cantidadEmpresa_y  \
0        1112248724         NjlD                  1               2318   
1        1112248724      MV6eM8N                  1               2318   
2        1112248724       5LzQkN                  1               2318   
3        1112248724      8M2rOBz                  1               2318   
4        1112248724      Dr6V3Da                  1               2318   
5        1112248724       8mkDQD                  1               2318   
6        1112248724      VNk6V22                  1               2318   
7        1112248724      dYjLleA                  1               2318   
8        1112248724      xkadArb                  1               2318   
9        1112248724      xkvx9D0                  1               2318   
10       1112248724        aj1v0                  1               2318   
11       1112248724        6OkGl                  1               2318   
12    

In [13]:
postulacionesOHC.columns

Index(['idaviso', 'idpostulante', 'datePostulacion', 'cantidadEmpresa',
       'zona_Capital Federal', 'zona_GBA Oeste', 'zona_Gran Buenos Aires',
       'tipo_Fines de Semana', 'tipo_Full-time', 'tipo_Part-time',
       'tipo_Pasantia', 'tipo_Por Contrato', 'tipo_Por Horas',
       'tipo_Primer empleo', 'tipo_Teletrabajo', 'tipo_Temporario',
       'tipo_Voluntario', 'nivel_Gerencia / Alta Gerencia / Dirección',
       'nivel_Jefe / Supervisor / Responsable', 'nivel_Junior', 'nivel_Otro',
       'nivel_Senior / Semi-Senior', 'sepostulo'],
      dtype='object')

In [11]:
avisosPostulados = postulacionesOHC[['idaviso','idpostulante','datePostulacion']]

In [73]:
avisosPostulados.info

<bound method DataFrame.info of             idaviso idpostulante datePostulacion
0        1112248724         NjlD      2018-01-19
1        1112248724      MV6eM8N      2018-01-30
2        1112248724       5LzQkN      2018-01-15
3        1112248724      8M2rOBz      2018-01-30
4        1112248724      Dr6V3Da      2018-01-23
5        1112248724       8mkDQD      2018-01-21
6        1112248724      VNk6V22      2018-01-15
7        1112248724      dYjLleA      2018-01-17
8        1112248724      xkadArb      2018-01-15
9        1112248724      xkvx9D0      2018-01-22
10       1112248724        aj1v0      2018-01-17
11       1112248724        6OkGl      2018-01-22
12       1112248724       686xdx      2018-01-24
13       1112248724      lDw9dxz      2018-01-15
14       1112248724      NzdJNoJ      2018-01-17
15       1112248724      8MaOpPD      2018-01-16
16       1112248724       aE15QW      2018-01-15
17       1112248724       E9jRRJ      2018-01-18
18       1112248724      mzVJE43     

In [15]:
avisosPostulados['cantPostulaciones'] = 1;
avisosPostuladosGP = avisosPostulados.groupby(['idaviso','idpostulante']).sum()
avisosPostuladosGP = avisosPostuladosGP.reset_index()
avisosPostuladosGP.info

<bound method DataFrame.info of             idaviso idpostulante  cantPostulaciones
0           8725750       1qwDrO                  1
1           8725750      2zkzE9Y                  1
2           8725750      2zwqd9V                  1
3           8725750      4rLbkk5                  1
4           8725750      4rPorjK                  1
5           8725750       5KQxVM                  1
6           8725750       606XOl                  1
7           8725750      6r5Ge3r                  1
8           8725750      6rB1NXx                  1
9           8725750      6rPGLR4                  1
10          8725750       6w8lOR                  1
11          8725750      8MP1qRz                  1
12          8725750       8r03az                  1
13          8725750       8rYAPO                  1
14          8725750       8vj3v3                  1
15          8725750      96XeV8P                  1
16          8725750      96Z5lP5                  1
17          8725750      96Zqw15

# Vistas

### Trabajo con la vistas, para reducirlas y generar avisos vistos, pero no postulados

In [15]:
vistas['timestamp'] = pd.to_datetime(vistas['timestamp'])
vistas['dateVista'] = pd.DatetimeIndex(vistas['timestamp']).date

In [17]:
vistasGrouped = vistas.groupby(['dateVista','idpostulante','idAviso']).count();
vistasGrouped = vistasGrouped.reset_index();

In [21]:
vistasGrouped.info

<bound method DataFrame.info of           dateVista idpostulante     idAviso  timestamp
0        2018-03-22      0z5JW1r  1112417455          1
1        2018-03-22      0zB02qr  1112317620          1
2        2018-03-22      0zB02qr  1112362627          2
3        2018-03-22      0zB02qr  1112384556          6
4        2018-03-22      0zB02qr  1112403061          2
5        2018-03-22      0zB02qr  1112410278          5
6        2018-03-22      0zB02qr  1112417329          2
7        2018-03-22      0zB1oA1  1112417467          1
8        2018-03-22      0zB2Kp9  1112408821          1
9        2018-03-22      0zB2bxr  1112246884          2
10       2018-03-22      0zB2bxr  1112352890          2
11       2018-03-22      0zB2bxr  1112355747          2
12       2018-03-22      0zB2bxr  1112407559          1
13       2018-03-22      0zB2bxr  1112409876          2
14       2018-03-22      0zB2eMa  1112362057          2
15       2018-03-22      0zB2eMa  1112419006          2
16       2018-03

In [39]:
#vistasGroupedFinal = vistasGrouped.drop('timeVista',axis=1)

In [ ]:
vistasGrouped['cantidadVistas'] = vistasGrouped['timestamp'];
vistasGrouped.drop('timestamp',axis=1,inplace=True)
vistasForMerge = vistasGrouped[['idpostulante','idAviso']]

In [121]:
def generoNOAvisos(cantParaArmar):
    sumaNoPostulados = pd.DataFrame();
    for i in range(int(cantParaArmar)):
        print("--------------------- Iteracion de sample " + str(i) +" de "+ str(cantParaArmar) +"---------");
        vistasSample = vistasForMerge.sample(frac=0.1);
        postulacionesSample = avisosPostuladosGP.sample(frac=0.1);

        avisosNoVistos = pd.merge(postulacionesSample,vistasSample,left_on='idaviso',right_on='idAviso',how='right', indicator=True)
        avisosNoVistosPosta = avisosNoVistos[avisosNoVistos['_merge'] == 'right_only']
#avisosNoVistosFinal = avisosNoVistosPosta.drop(['idpostulante_x','datePostulacion','cantPostulaciones','_merge'],axis=1)
        avisosNoVistosFinal = avisosNoVistosPosta.drop(['idpostulante_x','cantPostulaciones','_merge'],axis=1)
        avisosNoVistosFinal['idaviso'] = avisosNoVistosFinal['idAviso'];
        avisosNoVistosFinal['idpostulante'] = avisosNoVistosFinal['idpostulante_y'];
        avisosNoVistosFinal.drop(['idpostulante_y','idAviso'],axis=1,inplace=True);
        sumaNoPostulados = pd.concat([sumaNoPostulados,avisosNoVistosFinal])
    return sumaNoPostulados;




# Generamos archivos con avisos no vistos

In [134]:
import datetime as dt
def generoArchivosNOPostulado(cantArchivos,cantIteracionesPorArhivo):
    print("--- HORA DE INICIO PROCESO TOTAL "+ str(dt.datetime.now()) + "-----");
    for i in range(int(cantArchivos)):
        print("---------- Generando archivos " + str(i) +" de "+ str(cantArchivos) +"-------");
        print("---------- HORA DE INICIO "+ str(dt.datetime.now()) + "-------");
        sumaTotalNoAvisos = generoNOAvisos(int(cantIteracionesPorArhivo));
        sumaTotalNoAvisosSinDuplicados = sumaTotalNoAvisos.drop_duplicates().reset_index()
        sumaTotalNoAvisosSinDuplicados['sepostulo'] = 0
        sumaTotalNoAvisosSinDuplicados = sumaTotalNoAvisosSinDuplicados[['idpostulante','idaviso','sepostulo']]
        sumaTotalNoAvisosSinDuplicados.to_csv('nopostulados/nopostulados-'+str(i+10)+'.csv');
        print("---------- HORA DE FIN "+ str(dt.datetime.now()) + "--------------------------");
    print("--- HORA DE FIN PROCESO TOTAL "+ str(dt.datetime.now()) + "-----");

In [184]:
generoArchivosNOPostulado(5,30);

--- HORA DE INICIO PROCESO TOTAL 2018-06-19 21:22:08.105004-----
---------- Generando archivos 0 de 5-------
---------- HORA DE INICIO 2018-06-19 21:22:08.120486-------
--------------------- Iteracion de sample 0 de 30---------
--------------------- Iteracion de sample 1 de 30---------
--------------------- Iteracion de sample 2 de 30---------
--------------------- Iteracion de sample 3 de 30---------
--------------------- Iteracion de sample 4 de 30---------
--------------------- Iteracion de sample 5 de 30---------
--------------------- Iteracion de sample 6 de 30---------
--------------------- Iteracion de sample 7 de 30---------
--------------------- Iteracion de sample 8 de 30---------
--------------------- Iteracion de sample 9 de 30---------
--------------------- Iteracion de sample 10 de 30---------
--------------------- Iteracion de sample 11 de 30---------
--------------------- Iteracion de sample 12 de 30---------
--------------------- Iteracion de sample 13 de 30---------
-

--------------------- Iteracion de sample 3 de 30---------
--------------------- Iteracion de sample 4 de 30---------
--------------------- Iteracion de sample 5 de 30---------
--------------------- Iteracion de sample 6 de 30---------
--------------------- Iteracion de sample 7 de 30---------
--------------------- Iteracion de sample 8 de 30---------
--------------------- Iteracion de sample 9 de 30---------
--------------------- Iteracion de sample 10 de 30---------
--------------------- Iteracion de sample 11 de 30---------
--------------------- Iteracion de sample 12 de 30---------
--------------------- Iteracion de sample 13 de 30---------
--------------------- Iteracion de sample 14 de 30---------
--------------------- Iteracion de sample 15 de 30---------
--------------------- Iteracion de sample 16 de 30---------
--------------------- Iteracion de sample 17 de 30---------
--------------------- Iteracion de sample 18 de 30---------
--------------------- Iteracion de sample 19 de

In [145]:
nopostulados = nopostulados[['idpostulante','idaviso','sepostulo']];

In [148]:
avisosPostuladosGP['sepostulo'] = 1

In [186]:
avisosSample = avisosPostuladosGP.sample(frac=0.1)

In [271]:
datosMergeados = pd.concat([nopostulados,avisosSample])

In [272]:
datosMergeados = datosMergeados.fillna(0)

# Unimos con avisos con sus detalles

In [273]:
datosMergeadosConDetalle = pd.merge(datosMergeados,avisosDetalle,how='inner',on='idaviso');
datosMergeadosConDetalleFiltered = datosMergeadosConDetalle.drop(['titulo','descripcion','ciudad','mapacalle','nombre_area'],axis=1)

In [275]:
datosOHC = pd.concat([datosMergeadosConDetalleFiltered,pd.get_dummies(datosMergeadosConDetalleFiltered['nombre_zona'], prefix='zona')],axis=1)
datosOHC = pd.concat([datosOHC,pd.get_dummies(datosOHC['tipo_de_trabajo'], prefix='tipo')],axis=1)
datosOHC = pd.concat([datosOHC,pd.get_dummies(datosOHC['nivel_laboral'], prefix='nivel')],axis=1)

datosOHC.drop(['nivel_laboral','tipo_de_trabajo','nombre_zona'],axis=1,inplace=True)

In [276]:
datosOHC['denominacion_empresa'].fillna('S/E', inplace=True)
datosOHC = pd.merge(datosOHC,cantAvisosPorEmpresa,how='inner',on='denominacion_empresa')
datosOHC.drop('denominacion_empresa',axis=1,inplace=True);

In [ ]:
datosForTraining = datosOHC.drop(['sepostulo','idaviso','idpostulante'],axis=1)

# Probamos algoritmo

In [178]:
import pandas
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn import svm
from sklearn import tree
from sklearn import gaussian_process
from sklearn import neural_network

#Pre-procesamiento
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

In [194]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTraining, datosOHC['sepostulo'], test_size=0.7, random_state=7)

In [195]:
models = []
models.append(('RF10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
models.append(('RF20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
models.append(('RF50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
models.append(('RF60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('KNN 2', KNeighborsClassifier(n_neighbors=2)))
models.append(('KNN 3', KNeighborsClassifier(n_neighbors=3)))
models.append(('LR', linear_model.LinearRegression()))
models.append(('RIDGE.05', linear_model.Ridge (alpha = .5)))
models.append(('LASO.01', linear_model.Ridge (alpha = .1)))
models.append(('LASOLARS.05',linear_model.LassoLars (alpha = 0.5)))
models.append(('BAYESIAN',linear_model.BayesianRidge()))
models.append(('SVR',svm.LinearSVR()))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))
models.append(('MLP',tree.DecisionTreeRegressor()))
models.append(('NB', GaussianNB()))
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
 
TestModels = pd.DataFrame()
tmp = {}

results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, scoring='r2')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

RF10: 0.999965 (0.000003)
RF20: 0.999986 (0.000006)
RF50: 0.999991 (0.000004)
RF60: 0.999993 (0.000002)
KNN 2: 0.992723 (0.000696)
KNN 3: 0.991316 (0.000391)
LR: 0.999623 (0.000216)
RIDGE.05: 0.999623 (0.000215)
LASO.01: 0.999623 (0.000216)
LASOLARS.05: -0.000000 (0.000000)
BAYESIAN: 0.999623 (0.000216)
SVR: 0.999617 (0.000218)
DES-TREE: 1.000000 (0.000000)
MLP: 1.000000 (0.000000)
NB: 0.747843 (0.001135)
LR: -0.740777 (0.020453)


C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LDA: 1.000000 (0.000000)


C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


# Training Set

In [199]:
testingSet = pd.read_csv('resources/test_final_100k.csv')

In [200]:
testingSet.columns

Index(['id', 'idaviso', 'idpostulante'], dtype='object')

In [315]:
testingSet['testSet'] =1;
datosMergeados['testSet'] = 0;

In [316]:
datosMergeadosConDetalle = pd.merge(datosMergeados,avisosDetalle,how='inner',on='idaviso');
testingSetConDetalle = pd.merge(testingSet,avisosDetallesCompletos,how='left',on='idaviso');
#
#
datosConTestSet = pd.concat([datosMergeadosConDetalle,testingSetConDetalle]);
datosConTestSetFiltrado = datosConTestSet.drop(['titulo','descripcion','ciudad','mapacalle','nombre_area'],axis=1)

In [322]:
datosConTestOHC = pd.concat([datosConTestSetFiltrado,pd.get_dummies(datosConTestSetFiltrado['nombre_zona'], prefix='zona')],axis=1)
datosConTestOHC = pd.concat([datosConTestOHC,pd.get_dummies(datosConTestOHC['tipo_de_trabajo'], prefix='tipo')],axis=1)
datosConTestOHC = pd.concat([datosConTestOHC,pd.get_dummies(datosConTestOHC['nivel_laboral'], prefix='nivel')],axis=1)

datosConTestOHC.drop(['nivel_laboral','tipo_de_trabajo','nombre_zona'],axis=1,inplace=True)

In [323]:
datosConTestOHC['denominacion_empresa'].fillna('S/E', inplace=True)
datosConTestOHC = pd.merge(datosConTestOHC,cantAvisosPorEmpresa,how='left',on='denominacion_empresa')
datosConTestOHC.drop('denominacion_empresa',axis=1,inplace=True);

In [389]:
datosForTraining = datosConTestOHC[datosConTestOHC['testSet'] == 0]
datosForTest = datosConTestOHC[datosConTestOHC['testSet'] == 1]

In [329]:
datosForTrainingSinPostu = datosForTraining.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet'],axis=1)

In [330]:
datosForTestFinal = datosForTest.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet'],axis=1)

In [350]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
datosForTrainingSinPostuSinNulos = imp.fit_transform(datosForTrainingSinPostu)
datosForTestFinalSinNulos = imp.fit_transform(datosForTestFinal)

In [353]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingSinPostuSinNulos, datosForTraining['sepostulo'], test_size=0.7, random_state=7)

In [357]:
models = []
models.append(('RF10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
models.append(('RF20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
models.append(('RF50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
models.append(('RF60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('RF100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))
models.append(('KNN 2', KNeighborsClassifier(n_neighbors=2)))
models.append(('KNN 3', KNeighborsClassifier(n_neighbors=3)))
models.append(('LR', linear_model.LinearRegression()))
models.append(('RIDGE.05', linear_model.Ridge (alpha = .5)))
models.append(('LASO.01', linear_model.Ridge (alpha = .1)))
#models.append(('LASOLARS.05',linear_model.LassoLars (alpha = 0.5)))
models.append(('BAYESIAN',linear_model.BayesianRidge()))
models.append(('SVR',svm.LinearSVR()))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))
models.append(('MLP',tree.DecisionTreeRegressor()))
models.append(('NB', GaussianNB()))
#models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))



for name, model in models:
    m = model.fit(X_train, Y_train)
    pruebas = m.predict(datosForTestFinalSinNulos)
    archivo = pd.DataFrame()
    archivo['id'] = testingSet.loc[:,['id']]
    archivo['sepostulo'] = np.around(pruebas, decimals=0)
    archivo.to_csv('entregas/'+'prueba5-'+ name +'.csv', index=False)

C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


# Agregamos genero y Estudios

In [358]:
#Cargamos datasets

postulantesEducacion = pd.read_csv('resources/tar/fiuba_1_postulantes_educacion.csv')
postulantesGenero = pd.read_csv('resources/tar/fiuba_2_postulantes_genero_y_edad.csv')

postulantesEducacionZip = pd.read_csv('resources/zip/fiuba_1_postulantes_educacion.csv')
postulantesGeneroZip = pd.read_csv('resources/zip/fiuba_2_postulantes_genero_y_edad.csv')

In [397]:
educacionTodas = pd.concat([postulantesEducacion,postulantesEducacionZip])
generoTodas = pd.concat([postulantesGenero,postulantesGeneroZip])

In [398]:
import datetime as dt

generoTodas['fechanacimiento'] = pd.to_datetime(generoTodas['fechanacimiento'], format='%Y-%m-%d', errors='coerce')
generoTodas = generoTodas.dropna(subset=['fechanacimiento'])

hoy = pd.Timestamp(dt.datetime.now())
generoTodas['edad'] = (hoy - generoTodas['fechanacimiento']).astype('<m8[Y]')


In [399]:
generoTodasFiltered = generoTodas.loc[((generoTodas['edad'] >= 17) & (generoTodas['edad'] < 80 )), :]

In [400]:
postulantesGeneroOHC = pd.concat([generoTodasFiltered,pd.get_dummies(generoTodasFiltered['sexo'], prefix='sexo')],axis=1)
postulantesGeneroOHC = postulantesGeneroOHC[postulantesGeneroOHC['sexo_0.0'] == 0]
postulantesGeneroOHC = postulantesGeneroOHC[['idpostulante','sexo_FEM','sexo_MASC','sexo_NO_DECLARA','edad']]

postulantesGeneroOHC.drop_duplicates(subset='idpostulante',inplace=True)

# CAMBIAR FORMA DE TRABAJAR LA EDUCACION

In [401]:
postulantesEducacionOHC = pd.concat([educacionTodas,pd.get_dummies(educacionTodas['estado'], prefix='estado')],axis=1)
postulantesEducacionOHC = pd.concat([postulantesEducacionOHC,pd.get_dummies(postulantesEducacionOHC['nombre'], prefix='nombre')],axis=1)

postulantesEducacionOHC.drop(['nombre'],axis=1, inplace=True)
postulantesEducacionOHC.drop(['estado'],axis=1, inplace=True)

In [440]:
postulantesEducacionOHCGP = postulantesEducacionOHC.groupby('idpostulante').sum()
postulantesEducacionOHCGP = postulantesEducacionOHCGP.reset_index()

In [454]:
##CARGO LOS ELEMENTOS DEL TEST ANTERIOR
#datosForTraining
#datosForTest

datosForTraining = datosConTestOHC[datosConTestOHC['testSet'] == 0]
datosForTest = datosConTestOHC[datosConTestOHC['testSet'] == 1]

In [455]:
datosForTrainingConEducacion = pd.merge(datosForTraining,postulantesEducacionOHCGP,on='idpostulante',how='inner')
datosForTrainingConTodo = pd.merge(datosForTrainingConEducacion,postulantesGeneroOHC,on='idpostulante',how='inner')

In [456]:
datosForTestConEducacion = pd.merge(datosForTest,postulantesEducacionOHCGP,on='idpostulante',how='left')
datosForTestConTodo = pd.merge(datosForTestConEducacion,postulantesGeneroOHC,on='idpostulante',how='left')

In [457]:
datosForTestConTodo.info

<bound method DataFrame.info of        cantPostulaciones       id     idaviso  idpais idpostulante  sepostulo  \
0                    NaN      0.0      739260     1.0       6M9ZQR        NaN   
1                    NaN      1.0      739260     1.0       6v1xdL        NaN   
2                    NaN      2.0      739260     1.0       ezRKm9        NaN   
3                    NaN      3.0      758580     1.0       1Q35ej        NaN   
4                    NaN      4.0      758580     1.0       EAN4J6        NaN   
5                    NaN      5.0      758580     1.0       8R6pzR        NaN   
6                    NaN      6.0      776420     1.0       aZJ2XN        NaN   
7                    NaN      7.0      776420     1.0       Nmpo3J        NaN   
8                    NaN      8.0      776420     1.0       eVqWar        NaN   
9                    NaN      9.0      820850     1.0       6ZBD33        NaN   
10                   NaN     10.0      820850     1.0       1j4RNj        NaN

In [458]:
datosForTrainingSinPostu = datosForTrainingConTodo.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet'],axis=1)

In [459]:
datosForTestConTodoFiltrado = datosForTestConTodo.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet'],axis=1)

In [460]:
datosForTestConTodoFiltrado.columns.size

46

In [461]:
datosForTrainingSinPostu.columns.size

46

In [464]:
datosForTrainingSinPostu.drop('fechanacimiento',axis=1,inplace=True)

In [463]:
datosForTestConTodoFiltrado.drop('fechanacimiento',axis=1,inplace=True)

In [465]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
datosForTrainingSinPostuSinNulos = imp.fit_transform(datosForTrainingSinPostu)
datosForTestFinalSinNulos = imp.fit_transform(datosForTestConTodoFiltrado)

In [466]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingSinPostuSinNulos, datosForTrainingConTodo['sepostulo'], test_size=0.7, random_state=7)

In [469]:
models = []
#models.append(('RF10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
#models.append(('RF20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
#models.append(('RF50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
#models.append(('RF60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('RF100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))
#models.append(('KNN 2', KNeighborsClassifier(n_neighbors=2)))
#models.append(('KNN 3', KNeighborsClassifier(n_neighbors=3)))
#models.append(('LR', linear_model.LinearRegression()))
#models.append(('RIDGE.05', linear_model.Ridge (alpha = .5)))
#models.append(('LASO.01', linear_model.Ridge (alpha = .1)))
#models.append(('LASOLARS.05',linear_model.LassoLars (alpha = 0.5)))
#models.append(('BAYESIAN',linear_model.BayesianRidge()))
#models.append(('SVR',svm.LinearSVR()))
#models.append(('DES-TREE',tree.DecisionTreeRegressor()))
models.append(('MLP',tree.DecisionTreeRegressor()))
#models.append(('NB', GaussianNB()))
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))



for name, model in models:
    m = model.fit(X_train, Y_train)
    pruebas = m.predict(datosForTestFinalSinNulos)
    archivo = pd.DataFrame()
    archivo['id'] = testingSet.loc[:,['id']]
    archivo['sepostulo'] = np.around(pruebas, decimals=0)
    archivo.to_csv('entregas/'+'prueba6-'+ name +'.csv', index=False)